# Linear models and model selection

### **Part 1**: A lesson from de novo mutation (count) data

Let's imagine that we've sequenced 10 families, each with two parents and between 5-15 children. In each family, we've identified de novo mutations (DNMs) in the children.

For each child in each family, we also know the ages at conception for both parents. The goal of this analysis is to figure out whether there are "real" differences in parental age effects between families.

#### First, let's look at the contents of the DNM files.

Before we do any data analysis, we'll first want to convert the `fam_id` column to a "factor."

In [ ]:
df1 = read.csv('dnms_1.csv')
df2 = read.csv('dnms_2.csv')

df1$fam_id = as.factor(df1$fam_id)
df2$fam_id = as.factor(df2$fam_id)

head(df1)

#### Now, let's plot the data.

Here, we'll color each family's points (i.e., children) with a unique color.

In [ ]:
library(ggplot2)
library(cowplot)

ggplot(df1, aes(x=dad_age, y=dnms, color=fam_id)) +
geom_point(size=2) +
ylim(0,150)

Visually, there's a clear positive relationship between paternal age and the number of DNMs in the sample.

We can confirm this relationship by fitting a simple linear model.

#### Let's fit a simple linear model.

In [ ]:
model = lm(dnms ~ dad_age, data=df1)

We can also visualize the fit of the model.

In [ ]:
pred = predict(model, type="response", se.fit=TRUE)

ggplot(df1, aes(x=dad_age, y=dnms, color=fam_id)) +
geom_point(size=2) +
geom_line(data=cbind(df1, p=pred$fit), aes(x=dad_age, y=p), color='black')

#### What is the slope of this model? And is it significant?

By running `summary` on the fitted model, we can calculate the "beta," or coefficient associated with `dad_age`. This $\beta$ represents the expected increase in the number of DNMs per unit change in `dad_age`.

In [ ]:
print(summary(model))

The output of `summary` is also helpful, as it gives us the p-values associated with the calculated intercept and $\beta$ values.

For example, the p-value associated with the `dad_age` $\beta$ represents the probability that we'd see a $\beta$ value this extreme, given the null hypothesis that the $\beta =  0$. The actual p-value comes from a t-test as follows, where $s.e.$ is the standard error of the $\beta$ estimate:

$t = \frac{\beta} {s.e.(\beta)}$

This $t$ value should follow a $t$ distribution on $n - 2$ degrees of freedom *if the true $\beta = 0$*. Why $n - 2$? Well, we're estimating two parameters (slope and intercept) in our linear regression, so those things are unchanged in our t-test.

We can confirm the $t$ value reported in `summary` empirically:

In [ ]:
beta = summary(model)$coefficients[[2]]
stderr = summary(model)$coefficients[[4]]

t = beta / stderr
print(t)

#### Using ANOVA to assess model fit.

Instead of running `summary` on the fitted model, we could also run an analysis of variance (ANOVA). In this particular case, since we have only one independent variable, the ANOVA will tell us effectively the same thing as running `summary`.

In [ ]:
print(anova(model))

In practice, the `anova` is running an $F$-test that compares the the variance explained by the model and the "leftover," or "residual" variance *not* explained by the model. This is analogous to an ANOVA that compares within-group to between-group variance. 

In other words:

$$V_{model} = \sum_{n=i} (\hat{y_{i}} - \bar{y})^2$$

$$V_{residual} = \sum_{n=i} (y_{i} - \hat{y_{i}})^2$$

where, for a given point $i$,  $y_{i}$ represents the real, observed $y$ value, $\hat{y_{i}}$ represents the predicted value by the least-squares estimator, and $\bar{y}$ represents the mean of all $y$ values (the grand mean). In a "normal", one-way ANOVA, the $\hat{y_{i}}$ values would instead represent the group means.

The $F$ value then represents $\frac{V_{model}}{V_{residual}}$, and we check to see if the observed $F$ value follows an $F$ distribution with $k-1$ and $N-k$ degrees of freedom for the numerator and denominator, respectively. 

We can prove it by calculating all of this by hand.

In [ ]:
# add our predicted, "y-hat" values to the dataframe
df1$predictions = predict(model)

# calculate the V_model term
df1$mod = (df1$predictions - mean(df1$dnms)) ** 2
dof_mod = 1
v_model = sum(df1$mod) / dof_mod

# calculate the V_residual term
df1$resid = (df1$dnms - df1$predictions) ** 2
dof_resid = dim(df1)[[1]] - 2
v_resid = sum(df1$resid) / dof_resid

f = v_model / v_resid

print(f)

print(anova(model))

#### At its core, ANOVA is a method for comparing two models.

One last note about the ANOVA. If we run `anova(model)`, we're essentially testing whether the reduction in the residual sum of squares after adding a `dad_age` term to the model is significant. In other words, it's comparing two models:

```
m1 = lm(dnms ~ 1)
m2 = lm(dnms ~ dad_age)

anova(m1, m2)
```

And then running an $F$-test to compare `m1` and `m2`. We can prove it by running the above explicitly.

In [ ]:
m1 = lm(dnms ~ 1, data=df1)
m2 = lm(dnms ~ dad_age, data=df1)

print(anova(m1, m2))

As you can see, the results are exactly the same as in `anova(model)`. In the first line of output, we see that the RSS (residual sum of squares) in the first model is *all* of the variance, since Model 1 produces a flat line, with a slope of 0 and an intercept equal to the grand mean of all points. And that's why in the case of simple linear regression like this, the `anova` output will produce the same results as the `summary` output. 

Visually, the ANOVA (in the case of simple linear regression) is comparing the following two models and asking: does one of these models significantly reduce the residual sum of squares compared to the other?

In [ ]:
ggplot(df1, aes(x=dad_age, y=dnms, col=fam_id)) +
geom_point(size=2) +
geom_line(data=cbind(df1, p=pred$fit), aes(x=dad_age, y=p), col='dodgerblue') + # our linear regression model
geom_hline(yintercept=mean(df1$dnms), col="firebrick") # our null model (slope of 0, intercept at grand mean)


#### Adding terms to the model to better explain variance.

So far, we've been grouping all of our data together and fitting a single regression.

But by eye, you might be able to see subtle differences between the `dnms vs. age` relationships for each family.

In [ ]:
ggplot(df1, aes(x=dad_age, y=dnms, color=fam_id)) +
facet_wrap(~fam_id, nrow=5) +
geom_point(size=2)

If we hypothesize that there are, indeed, differences between the slopes and intercepts of each family's regressions, can we model these differences between families? One option is to fit a new model that considers differences in slopes and intercepts between each family's regression.

In [ ]:
model_interaction = lm(dnms ~ dad_age + fam_id + dad_age:fam_id, data=df1)

This new model adds two terms: `fam_id` and `dad_age:fam_id`. The `fam_id` term is called an "additive" term, and `dad_age:fam_id` is called an "interaction" term. Let's look at the output of the `summary` function.

In [ ]:
print(summary(model_interaction))

As you can see, the summary output is quite a bit more complicated now. What `R` has done is choose the first family, `fam_id0`, and fit a regression to that family. The least-squares result of that regression produced a line of best fit with a slope of 1.24551 and an intercept of 12.80475. `R` has then fit a regression for every other family, producing a unique intercept and slope value for that family. The numbers in the `estimate` column of the `summary` output represent how much you'd need to "shift" the slope and intercept calculated for `fam_id0` to get the slope and intercept for every other family.

As you might be able to tell (simply based on the magnitude of the numbers), the `estimate` values for `fam_id1`, `fam_id2`, etc. represent the "shifts" for the intercept in that family, and the `estimate` values for `dad_age:fam_id1` represent the "shifts" needed for the slope in that family.

When running a model like this, `R` needs to choose one group (i.e., family) to be the baseline. By default, `R` will choose the first group in your dataframe, which is why `fam_id0` is the baseline here. You can change which baseline group `R` uses as follows:



In [ ]:
df1$fam_id = relevel(df1$fam_id, ref = 6)

model_interaction = lm(dnms ~ dad_age + fam_id + dad_age:fam_id, data=df1)

print(summary(model_interaction))

As you can see, `fam_id0` is no longer the baseline, having been swapped for `fam_id6`.

For each $x$ value, we'd normally predict its $y$ value by simply using the global slope and intercept values we calculated in our first model.

Now, what we're doing is: for each $x$ value, we can now ask: which family (i.e., group) did that $x$ value come from? Then, we'll use the slope and intercept calculate *for that family* to predict the $y$ value.

We can also test whether adding these new terms to our model actually improves its fit to the underlying data using an ANOVA.



In [ ]:
print(anova(model_interaction))

The ANOVA table is now telling us how significant each of the terms is. In other words, do we significantly reduce the residual sum of squares by adding each term to our model? 

In practice, this is like comparing models three separate times:

```
m1 = dnms ~ 1
m2 = dnms ~ dad_age
m2 = dnms ~ dad_age + fam_id
m4 = dnms ~ dad_age + fam_id + dad_age:fam_id

anova(m1, m2)
anova(m2, m3)
anova(m3, m4)
```

NOTE: the way the ANOVA works means that terms are added sequentially. This means that more and more variance is explained each time we add a term, and by the time we add the last term (the interaction term), much of the variance has already been "soaked up" by the first two terms. And `R` *always* adds interaction terms last.

In this example, we see that adding a `fam_id` term improves the fit of the model, but adding the `dad_age:fam_id` term on top of that doesn't produce a significantly better fit. We can interpret this as meaning that the intercepts significantly differ between families, but slopes do not. This makes sense, sense I simulated the data such that each family has approximately the same slope, but different intercepts.

Visually, we can interpret all of these different models in the following way.

#### Visualizing residuals from a simple linear regression

Since we're predicting y values using a single slope and intercept, regardless of family membership, all of the predicted points follow a straight line.

In [ ]:
m1 = lm(dnms ~ dad_age, data=df1)

df1$predictions = predict(m1)
df1$residuals = residuals(m1)

p1 <- ggplot(df1, aes(x=dad_age, y=dnms)) +
    geom_segment(aes(xend = dad_age, yend = predictions)) +
    geom_point(pch=21, col="black",  size=2.5) +
    geom_point(aes(y = predictions), pch=21, fill="firebrick", size=2.5, col="white") 

p1

#### Visualizing residuals from a model with interaction

Now, the predicted values are much closer to the "real" values. But since we've adjusted each family's points by a family-specific slope and intercept, the predicted values no longer follow one single line.

In [ ]:
m2 = lm(dnms ~ dad_age * fam_id, data=df1)

df1$predictions = predict(m2)
df1$residuals = residuals(m2)

p2 <- ggplot(df1, aes(x=dad_age, y=dnms)) +
    geom_segment(aes(xend = dad_age, yend = predictions)) +
    geom_point(pch=21, col="black",  size=2.5) +
    geom_point(aes(y = predictions), pch=21, fill="firebrick", size=2.5, col="white") 

p2

#### Visualizing our model fit to each family separately

In [ ]:
m2 = lm(dnms ~ dad_age * fam_id, data=df1)

#calculate confidence bands
cb = predict(m2, int="c", newdata=df1)

# add confidence bands to dataframe
df_cb = cbind(df1, cb)

ggplot(df_cb, aes(x=dad_age, y=dnms, color=fam_id)) +
facet_wrap(~fam_id, nrow=5) +
geom_line(aes(x=dad_age, y=predictions)) +
geom_ribbon(aes(x=dad_age, ymin=lwr, ymax=upr), color='grey', alpha=0.25) +
geom_point(size=2)